### Multivariate time series prediction using MLP


Here is a pytorch implementation of a fully connected neural network which has in total 4 fully connected layers. This is a vanilla
neural network model to be used for multi variate time series prediction. The data we have is 2D, the model learns first
along the time axis and secondly it learns along the feature axis. It has two fully connected layers with size
**seq_len x 100** and **100 x pred_len** with **ReLU** loss function along time axis and two fully connected layers with sizes **num_features x 20** and **20 x 1** with same **ReLU** as loss function along the features axis. The neural network uses Mean Square Error Loss (MSELoss) to calculate the model loss. NSE and WAPE are used for model evaluation. The data is normalized using StandardScaler from scikit learn. Data is used is the 3 hr river gauge height data. It is divided in the ratio of 7:1:2 training, validation and testing respectively.

This notebooks implements MLP with **regularization and early stopping** with model implemented to fully use **GPU**.

In [1]:
import torch
import numpy as np

import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [2]:
path = '../dataset/final_data.csv'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Implement determinism. Set a fixed value for random seed so that when the parameters are initialized, they are initialized same across all experiments.
torch.manual_seed(42)

# If you are using CUDA, also set the seed for it
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Set the seed for NumPy
np.random.seed(42)

Using device: cpu


Here we define **RiverData** a custom Dataset class to load the dataset we have. It extends the pytorch Dataset class.  
- We need to define \_\_init__() function which can be used for loading data from file and optionally for data preprocessing.
- Thereafter we define \_\_len__() function which gives the length of dataset.
- Then we define \_\_getitem__() function which returns an instance of (feature, label) tuple which can be used for model training.
  For our time series data, feature means the past values to be used for training and label means the future values to be predicted.

In [4]:
class RiverData(torch.utils.data.Dataset):
    
    def __init__(self, df, target, datecol, seq_len, pred_len):
        self.df = df
        self.datecol = datecol
        self.target = target
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.setIndex()
        

    def setIndex(self):
        self.df.set_index(self.datecol, inplace=True)
    

    def __len__(self):
        return len(self.df) - self.seq_len - self.pred_len


    def __getitem__(self, idx):
        if len(self.df) <= (idx + self.seq_len+self.pred_len):
            raise IndexError(f"Index {idx} is out of bounds for dataset of size {len(self.df)}")
        df_piece = self.df[idx:idx+self.seq_len].values
        feature = torch.tensor(df_piece, dtype=torch.float32)
        label_piece = self.df[self.target][idx + self.seq_len:  idx+self.seq_len+self.pred_len].values
        label = torch.tensor(label_piece, dtype=torch.float32)
        return (feature.T, label) 

### Normalize the data

In [5]:
df = pd.read_csv(path)
df = df[df['DATE'] > '2012']

raw_df = df.drop('DATE', axis=1, inplace=False)
scaler = MinMaxScaler()

# Apply the transformations
df_scaled = scaler.fit_transform(raw_df)

df_scaled = pd.DataFrame(df_scaled, columns=raw_df.columns)
df_scaled['DATE'] = df['DATE']
df = df_scaled

Some advanced python syntax have been used here. \
*common_args : it's used to pass arguments to a function, where common_args represents a python list \
**common_args: it's used to pass arguments to a function, where common_args represents a python dictionary

In [6]:

train_size = int(0.7 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

seq_len = 13
pred_len = 1
num_features = 7

common_args = ['gauge_height', 'DATE', seq_len, pred_len]
train_dataset = RiverData(df[:train_size], *common_args)
val_dataset = RiverData(df[train_size: train_size+val_size], *common_args)
test_dataset = RiverData(df[train_size+val_size : len(df)], *common_args)


In [7]:
# Important parameters

BATCH_SIZE = 512 # keep as big as can be handled by GPU and memory
SHUFFLE = False # we don't shuffle the time series data
DATA_LOAD_WORKERS = 1 # it depends on amount of data you need to load


In [8]:
from torch.utils.data import DataLoader

common_args = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE}
train_loader = DataLoader(train_dataset, **common_args)
val_loader = DataLoader(val_dataset, **common_args)
test_loader = DataLoader(test_dataset, **common_args)

### Here we define our pytorch model.

BasicMLPNetwork is the model class, it extends the Module class provided by pytorch. \
- We define \_\_init__() function. It sets up layers and defines the model parameters.
- Also, we define forward() function which defines how the forwared pass computation occurs

In [9]:
# Here we are adding dropout layers.

class BasicMLPNetwork(torch.nn.Module):
    
    def __init__(self, seq_len, pred_len, num_features, dropout):
        # call the constructor of the base class
        super().__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.num_features = num_features
        hidden_size_time = 512
        hidden_size_feat = 128
        # define layers for combining across time series
        self.fc1 = torch.nn.Linear(self.seq_len, hidden_size_time)
        self.relu = torch.nn.ReLU()
        self.dropout1 = torch.nn.Dropout(p=dropout)
        self.fc2 = torch.nn.Linear(hidden_size_time, self.pred_len)
        self.dropout2 = torch.nn.Dropout(p=dropout)

        # define layers for combining across the features
        self.fc3 = torch.nn.Linear(self.num_features, hidden_size_feat)
        self.fc4 = torch.nn.Linear(hidden_size_feat, 1)

    def forward(self, x):

        # computation over time
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = self.relu(out) # has dimension 512 x 7 x 12
        out = self.dropout2(out)
        # computation over features
        out = out.transpose(1,2) # dimension 512 x 12 x 7
        out = self.fc3(out) # dimension 512 x 12 x 20
        out = self.relu(out)
        out = self.fc4(out) # dimension 512 x 12 x 1

        out = out.squeeze(-1) # dimension 512 x 12
        
        return out

# Note that the gradients are stored insize the FC layer objects
# For each training example we need to get rid of these gradients

In [10]:
dropout_p = 0.189369
learning_rate = 0.000294
weight_decay = 0.00056  # Regularization strength

model = BasicMLPNetwork(seq_len, pred_len, num_features, dropout_p)
model = model.to(device)
loss = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)

In [11]:
for gen in model.parameters():
    print(gen.shape)

torch.Size([512, 13])
torch.Size([512])
torch.Size([1, 512])
torch.Size([1])
torch.Size([128, 7])
torch.Size([128])
torch.Size([1, 128])
torch.Size([1])


In [12]:
for i, (f,l) in enumerate(train_loader):
    print('features shape: ', f.shape)
    print('labels shape: ', l.shape)
    break

features shape:  torch.Size([512, 7, 13])
labels shape:  torch.Size([512, 1])


In [13]:
# define metrics
import numpy as np
epsilon = np.finfo(float).eps

def Wape(y, y_pred):
    """Weighted Average Percentage Error metric in the interval [0; 100]"""
    y = np.array(y)
    y_pred = np.array(y_pred)
    nominator = np.sum(np.abs(np.subtract(y, y_pred)))
    denominator = np.add(np.sum(np.abs(y)), epsilon)
    wape = np.divide(nominator, denominator) * 100.0
    return wape

def nse(y, y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)
    return (1-(np.sum((y_pred-y)**2)/np.sum((y-np.mean(y))**2)))


def evaluate_model(model, data_loader):
    # following line prepares the model for evaulation mode. It disables dropout and batch normalization if they have 
    # are part of the model. For our simple model it's not necessary. Still I'm going to use it.

    model.eval()
    all_inputs = torch.empty((0, num_features, seq_len))
    all_labels = torch.empty(0, pred_len)
    for inputs, labels in data_loader:
        all_inputs = torch.vstack((all_inputs, inputs))
        all_labels = torch.vstack((all_labels, labels))
    
    with torch.no_grad():
        all_inputs = all_inputs.to(device)
        outputs = model(all_inputs).detach().cpu()
        avg_val_loss = loss(outputs, all_labels)
        nsee = nse(all_labels.numpy(), outputs.numpy())
        wapee = Wape(all_labels.numpy(), outputs.numpy())
        
    print(f'NSE : {nsee}', end=' ')
    print(f'WAPE : {wapee}', end=' ')
    print(f'Validation Loss: {avg_val_loss}')
    model.train()
    return avg_val_loss


In [14]:
num_epochs = 200
best_val_loss = float('inf')
patience = 5

for epoch in range(num_epochs):
    epoch_loss = []
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss_val = loss(outputs, labels)

        # calculate gradients for back propagation
        loss_val.backward()

        # update the weights based on the gradients
        optimizer.step()

        # reset the gradients, avoid gradient accumulation
        optimizer.zero_grad()
        epoch_loss.append(loss_val.item())

    avg_train_loss = sum(epoch_loss)/len(epoch_loss)
    print(f'Epoch {epoch+1}: Traning Loss: {avg_train_loss}', end=' ')
    avg_val_loss = evaluate_model(model, val_loader)

    # Check for improvement
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print('Early stopping!')
            # Load the best model before stopping
            model.load_state_dict(torch.load('best_model.pth'))
            break

Epoch 1: Traning Loss: 0.01744051495857612 NSE : -0.0004334449768066406 WAPE : 41.44640688705754 Validation Loss: 0.013194818049669266
Epoch 2: Traning Loss: 0.01723201509921393 NSE : -0.0007778406143188477 WAPE : 41.38129416191354 Validation Loss: 0.01319936104118824
Epoch 3: Traning Loss: 0.017206342720532876 NSE : -0.001185297966003418 WAPE : 41.32305981043181 Validation Loss: 0.013204734772443771
Epoch 4: Traning Loss: 0.017190685550425976 NSE : -0.001508951187133789 WAPE : 41.284773288040434 Validation Loss: 0.013209004886448383
Epoch 5: Traning Loss: 0.01717302050510259 NSE : -0.0019409656524658203 WAPE : 41.24069933150499 Validation Loss: 0.013214699923992157
Epoch 6: Traning Loss: 0.01714886643536289 NSE : -0.0027016401290893555 WAPE : 41.17602570951706 Validation Loss: 0.013224733993411064
Early stopping!


In [15]:
evaluate_model(model, test_loader)

NSE : -0.1523803472518921 WAPE : 58.526007822577576 Validation Loss: 0.01285931933671236


tensor(0.0129)